In [49]:
import sklearn 
import pandas
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score,train_test_split

In [2]:
X,y = make_classification(n_samples=10000,)
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Evaluation

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
cross_val_score(
    LogisticRegression(multi_class='multinomial'),
    X_train,
    y_train,cv=3
)

array([0.90926134, 0.92388451, 0.92460615])

In [5]:
from sklearn.neighbors import KNeighborsClassifier

In [6]:
cross_val_score(
    KNeighborsClassifier(n_neighbors=5),
    X_train,
    y_train,cv=3
)

array([0.8927634 , 0.90101237, 0.89797449])

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
cross_val_score(
    RandomForestClassifier(n_estimators=5),
    X_train,
    y_train,cv=3
)

array([0.90663667, 0.91301087, 0.91297824])

In [9]:
from sklearn.neural_network import MLPClassifier

In [10]:
cross_val_score(
    MLPClassifier(max_iter=100),
    X_train,
    y_train,cv=3
)

/home/rahmani/work/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/rahmani/work/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/rahmani/work/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


array([0.90888639, 0.91788526, 0.90997749])

In [11]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
clf4 = SVC()
clf5 = MLPClassifier()

voting_clf = VotingClassifier([('lgr', clf1),    ('rf', clf2),    ('nb', clf3),('svm', clf4),('mlp', clf5)],
    voting='hard')

In [12]:
voting_clf.fit(X_train,y_train)

/home/rahmani/work/envs/ML/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


VotingClassifier(estimators=[('lgr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('rf',
                              RandomForestClassifier(n_estimators=50,
                                                     random_state=1)),
                             ('nb', GaussianNB()), ('svm', SVC()),
                             ('mlp', MLPClassifier())])

In [13]:
voting_clf.score(X_test,y_test)

0.9395

In [14]:
estimator = ['rf','nb','svm','mlp']
for est in estimator:
    
    print( "estimator= ", est, 'Score= ', voting_clf.named_estimators_[est].score(X_test,y_test))

estimator=  rf Score=  0.9405
estimator=  nb Score=  0.937
estimator=  svm Score=  0.934
estimator=  mlp Score=  0.9225


In [15]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [16]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=10,max_samples=100,n_jobs=-1, random_state=0, oob_score=True ).fit(X_train,y_train)

In [17]:
bag_clf.oob_score_

0.913875

In [18]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9285

In [27]:
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

In [30]:
clf6 = KNeighborsClassifier()

stacking = StackingClassifier(estimators=[
    # ('rf', clf2),
    # ('xgb', clf3),
    # ('svr', clf4),
    # ('mlp', clf5),
    ('knn',clf6),
    # ('gbrt' , clf7)
],
final_estimator=SVC(random_state=43),
cv=3,
n_jobs=-1) 
stacking

StackingClassifier(cv=3, estimators=[('knn', KNeighborsClassifier())],
                   final_estimator=SVC(random_state=43), n_jobs=-1)

In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix


In [69]:
model = Pipeline([('stack',stacking)])
hyper = {"stack__knn__n_neighbors":[1,3,5,7], 'stack__final_estimator__C':[1.0,0.75,0.5,0.25], 
         'stack__final_estimator__kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']}


clf = GridSearchCV(model, param_grid=hyper , cv=3,scoring='f1_micro')
clf.fit(X_train,y_train)

/home/rahmani/work/envs/ML/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
36 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "/home/rahmani/work/envs/ML/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/rahmani/work/envs/ML/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rahmani/work/envs/ML/lib/python3.11/site-packages/sklearn/pipelin

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('stack',
                                        StackingClassifier(cv=3,
                                                           estimators=[('knn',
                                                                        KNeighborsClassifier())],
                                                           final_estimator=SVC(random_state=43),
                                                           n_jobs=-1))]),
             param_grid={'stack__final_estimator__C': [1.0, 0.75, 0.5, 0.25],
                         'stack__final_estimator__kernel': ['linear', 'poly',
                                                            'rbf', 'sigmoid',
                                                            'precomputed'],
                         'stack__knn__n_neighbors': [3, 5, 7]},
             scoring='f1_micro')

In [70]:
y_predict = clf.predict(X_test)

In [71]:
confusion_matrix(y_test,y_predict)

array([[924, 108],
       [ 55, 913]])

In [72]:
from sklearn.metrics import confusion_matrix, classification_report

In [73]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      1032
           1       0.89      0.94      0.92       968

    accuracy                           0.92      2000
   macro avg       0.92      0.92      0.92      2000
weighted avg       0.92      0.92      0.92      2000



In [74]:
pd.DataFrame(clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_stack__final_estimator__C,param_stack__final_estimator__kernel,param_stack__knn__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.159964,0.009588,0.052274,0.002375,1.0,linear,3,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.886014,0.832771,0.888972,0.869252,0.025825,29
1,0.133735,0.003447,0.049677,0.003932,1.0,linear,5,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.892763,0.901012,0.897974,0.897250,0.003406,9
2,0.133952,0.002000,0.049590,0.005877,1.0,linear,7,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.899513,0.903262,0.904726,0.902500,0.002196,1
3,0.275966,0.035030,0.061937,0.001688,1.0,poly,3,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.838770,0.832771,0.832333,0.834625,0.002937,37
4,0.191377,0.007467,0.054145,0.002488,1.0,poly,5,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.880390,0.881140,0.872843,0.878124,0.003747,25
5,0.211338,0.028988,0.054287,0.002137,1.0,poly,7,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.890889,0.889764,0.891223,0.890625,0.000624,17
6,0.170601,0.010448,0.089737,0.004583,1.0,rbf,3,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.886014,0.896513,0.888972,0.890500,0.004420,21
7,0.190692,0.017375,0.086767,0.003384,1.0,rbf,5,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.892763,0.901012,0.897974,0.897250,0.003406,9
8,0.180006,0.009451,0.081631,0.003686,1.0,rbf,7,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.899513,0.903262,0.904726,0.902500,0.002196,1
9,0.295017,0.004439,0.111499,0.002065,1.0,sigmoid,3,"{'stack__final_estimator__C': 1.0, 'stack__fin...",0.831646,0.835396,0.837959,0.835000,0.002593,33


In [75]:
predict = clf.best_estimator_.predict(X_test)

In [76]:
from sklearn import metrics

metrics.r2_score(y_test,predict)

0.6736658338138253